<a href="https://colab.research.google.com/github/FelipeNemo/email-validation-guide/blob/syntax-verifier/emailvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sistema de Validação de Email para prevenção de fraude**

**Autores:** Felipe Augusto Batista Mendes dos Santos



Este sistema atende à necessidade de validação de e-mails para aumentar a eficiência e a segurança das transações online, utilizando um pipeline de processamento. Ele visa usar o status de classificação de validade para a tomada de decisões.

* **Input:** Dados estruturados
* **Output:** Status de classificação de e-mail

# Verificação de Email
Verificação de um endereço de email existente e ativo, ou seja, válido. Um email válido pode receber mensagens de outros remetentes. São responsáveis por executar uma etapa do **PROCESSO DE VERIFICAÇÃO**, como por exemplo:
* Verificações de sintaxe
* Domínio
* Catch-all
* Registro MX
* Ping de endereço de e-mail

# Validação de email
É o resultado final do processo de verificação de email. Às vezes, a validação também implica na verificação de qualquer informação adicional que vem com o e-mail do lead ou na verificação da origem desse lead.

* “Não verificado” ⭕ - antes do processo de verificação.

* “Válido” 🟢 – significa que um e-mail está correto e existe.

* “Inválido” 🟡 -  significa que esse endereço de e-mail pode receber mensagens, mas devido a uma configuração em todo o domínio, não é possível determinar a sua validade real, assim ele é marcado como Não verificável. Esses e-mails também são conhecidos como e-mails “pegam tudo” ou “aceitam tudo”.

* “Não verificável” 🔴 – significa que um endereço de e-mail não existe ou não está mais ativo.

*  “Não é possível encontrar o e-mail” ⚪ - clientes potenciais com informações pessoais, como fotos ou experiência profissional etc., no entanto, não há endereço de e-mail.

# Para que verificar emails?

1.    Manter listas de e-mail atualizadas
2.    Reduz hard bounces
3.    Melhora a capacidade de entrega de e-mail
4.    Mantém a reputação do remetente elevada
4.    Prevenção de ataque de fraude










# Como funciona um verificador de e-mail:
1.   Verificação de sintaxe
2.   Verificação de jargão
3.   Verificação de existência de domínio
4.   Verificação de registro MX
5.   Verificação de domínio catch-all
6.   Autenticação SMTP (ping de endereço de e-mail)
7.   Verificação de freemail




# Leitura .csv

# 1. Verificação de sintaxe:
**Ferramentas:**  

*   Python

## Correspondência de padrão de email:

> < local-part@domain >

1. **local-part** -  Pode ter até 64 octetos de comprimento, disposição de pontos e caracteres especiais.

2. **@** - Deve conter apenas 1 elemento "@"em todo o e-mail

3. **domain** -  Pode ter no máximo 255 octetos e deve conter apenas 1 "."

  Exemplo: .org .net .com



In [14]:
def verificador_global(email):
      # Verifica se contém exatamente um "@"
    if email.count('@') != 1:
        return False

    local, domain = email.split('@')

    return True

### Local-part:
*  64 octetos de comprimento

* **Disposição geral de pontos:**

 Não é permitido que seja o primeiro ou o último caractere, e que também apareça consecutivamente

* **Caracteres especiais:**

> "() , : ; < > @ [ \ ]

  -  São permitidos com restrições(somente dentro de uma string entre aspas e nessa string entre aspas, qualquer barra invertida ou aspas duplas deve ser precedida uma vez por uma barra invertida)
   
   Exemplo:
   
   john.smith(comment)@example.come e (comment)john.smith@example.com são ambos equivalentes a john.smith@example.com

In [15]:
#Constantes:
CARACTERES_ESPECIAIS = { "(", ")", ",", ":", ";", "<", ">", "@", "[", "]" }

In [16]:
def local_sintax(local, verificador_global):

    # Verifica o comprimento da parte local
    if len(local.encode('utf-8')) > 64:
        return False

    # Verifica a disposição de pontos na parte local
    if local.startswith('.') or local.endswith('.'):
        return False
    if '..' in local:
        return False

    # Verifica a presença de caracteres especiais na parte local
    if any(char in CARACTERES_ESPECIAIS for char in local):
        if not (local.startswith('"') and local.endswith('"')):
            return False
        for i, char in enumerate(local):
            if char in CARACTERES_ESPECIAIS:
                if i == 0 or i == len(local) - 1 or local[i - 1] != '\\':
                    return False
    return True


### Domain
* 255 octetos de comprimento
* **Disposição geral de pontos:**

  Deve conter apenas 1 "." :

In [ ]:
def domain_sintax(domain, verificador_global)
    # Verifica o comprimento da parte do domínio
    if len(domain.encode('utf-8')) > 255:
        return False

    # Verifica se o domínio contém exatamente um "."
    if domain.count('.') != 1:
        return False

### Conclusão:

In [ ]:
def verificador_sintax(local_sintax, domain_sintax):
  pass

# 2. Verificação de domínio:
Ferramentas:

*   DNSRECON


# 3.Verificação catch-all

# 4. Verificação do registro MX:
Ferramentas:

*   Pydantic

# 5. Ping do endereço de e-mail

# Futuras melhorias:


## Referências:
* Principal Artigo de referência:

 https://snov.io/blog/br/verificacao-de-email-explicada/
*   Plataforma de validação de email que inspirou o projeto:

 https://snov.io/br/verificador-de-emails
*  Entendendo DNS:

 https://support.google.com/a/answer/48090?hl=pt-BR#:~:text=voltar%20ao%20in%C3%ADcio-,Registro%20MX,de%20e%2Dmail%20do%20Google.
*   Status de email:

 https://snov.io/knowledgebase/email-statuses/

*   Definições de leads:

 https://fleeg.com/blog/inbound-marketing/leads/?origin=fleeg_snovio_link_backlink&utm_source=Snovio&utm_medium=link&utm_campaign=backlink
*   Verificação sintaxe:

 https://en.wikipedia.org/wiki/Email_address#Local-part

 https://en.wikipedia.org/wiki/Octet_(computing)

 https://www.youtube.com/watch?v=WkNBhYMUXwI
*   Envio de email SMTP:

 https://www.youtube.com/watch?v=umvzsQLZYD4
*   Verificação TLD/MX record/Sintaxe:

 https://www.youtube.com/watch?v=Bj61jJo2Mvs
*   Item da lista
*   Item da lista



